In [26]:
import pandas as pd
def print_ratio(df,x_values,save_name, last_mode=None):
    # 计算每个组合的平均return
    avg_return = df.groupby(['algorithm', 'train mode', 'test mode'])['ep ret'].mean().reset_index()

    result_df = pd.DataFrame(columns=['algorithm'])
    algorithms = avg_return['algorithm'].unique()

    for algo in algorithms:
        algo_data = avg_return[avg_return['algorithm'] == algo]
        
        algo_ratios = {'algorithm': algo}
        for x in x_values:
            before_condition = (avg_return['train mode'] == x) & (avg_return['test mode'] == x)
            max_ret = avg_return[before_condition]['ep ret'].max()
            
            # condition after training on new task
            after_condition = (algo_data['train mode'] == last_mode) & (algo_data['test mode'] == x)
            after_resutls = algo_data[after_condition]
            
            # condition before training on new task
            before_condition = (algo_data['train mode'] == x) & (algo_data['test mode'] == x)
            before_results = algo_data[before_condition]
            
            if not after_resutls.empty and not before_results.empty:
                performance_before = before_results['ep ret'].values[0] / max_ret
                performance_after = after_resutls['ep ret'].values[0] / max_ret
                ratio = round(performance_before - performance_after, 2)
                # ratio = (performance_before - performance_after) / performance_before
                # ratio = performance_before - performance_after
                algo_ratios[f'forgetting on {x}'] = ratio 
            else:
                algo_ratios[f'forgetting on {x}'] = None  
        
        # algo_ratios_df = pd.DataFrame(algo_ratios)
        # result_df = pd.concat([result_df, algo_ratios_df], ignore_index=True)
        result_df = result_df.append(algo_ratios, ignore_index=True)

    # print(result_df)
    numeric_df = result_df.select_dtypes(include='number')

    result_df['mean'] = round(numeric_df.mean(axis=1),2)
    # print(resul)
    result_df.to_csv(f'{save_name}.csv', index=False)
    print(result_df)
    
def print_final_performance(df,x_values,save_name):
    # 计算每个组合的平均return
    avg_return = df.groupby(['algorithm', 'train mode', 'test mode'])['ep ret'].mean().reset_index()

    result_df = pd.DataFrame(columns=['algorithm'])
    algorithms = avg_return['algorithm'].unique()
    train_mode = x_values[-1]
    for algo in algorithms:
        algo_data = avg_return[avg_return['algorithm'] == algo]
        
        algo_ratios = {'algorithm': algo}
        # 遍历每个x值
        for x in x_values:
            cur_condition = (avg_return['train mode'] == x) & (avg_return['test mode'] == x)
            max_ret = avg_return[cur_condition]['ep ret'].max()
            
            # 筛选出train mode = x, test mode = x-1的行
            condition1 = (algo_data['train mode'] == train_mode) & (algo_data['test mode'] == x)
            result1 = algo_data[condition1]
            
            # 筛选出train mode = x-1, test mode = x-1的行
            prev_condition = (algo_data['train mode'] == x) & (algo_data['test mode'] == x)
            result2 = algo_data[prev_condition]
            
            # 如果两组数据都存在，则计算比值
            if not result1.empty and not result2.empty:
                ratio = result1['ep ret'].values[0] / max_ret
                algo_ratios[f'{x}'] = ratio  # 动态列名
            else:
                algo_ratios[f'{x}'] = 0  # 如果没有匹配的数据，则设为None
        
        result_df = result_df.append(algo_ratios, ignore_index=True)

    # print(result_df)
    numeric_df = result_df.select_dtypes(include='number')

    result_df['mean'] = numeric_df.mean(axis=1)
    # print(resul)
    result_df.to_csv(f'{save_name}.csv', index=False)
    print(result_df)
import pandas as pd
def print_ratio_MetaWorld(df,x_values,save_name):
    # 计算每个组合的平均return
    avg_success = df.groupby(['algorithm', 'train task', 'test task'])['success'].mean().reset_index()

    result_df = pd.DataFrame(columns=['algorithm'])
    algorithms = avg_success['algorithm'].unique()
    train_mode = x_values[-1]
    for algo in algorithms:
        algo_data = avg_success[avg_success['algorithm'] == algo]
        
        algo_ratios = {'algorithm': algo}
        # 遍历每个x值
        for x in x_values:
            # cur_condition = (avg_success['train task'] == x) & (avg_success['test task'] == x)
            
            # 筛选出train mode = x, test mode = x-1的行
            condition1 = (algo_data['train task'] == train_mode) & (algo_data['test task'] == x)
            after_result = algo_data[condition1]
            
            # 筛选出train mode = x-1, test mode = x-1的行
            prev_condition = (algo_data['train task'] == x) & (algo_data['test task'] == x)
            before_result = algo_data[prev_condition]
            
            # 如果两组数据都存在，则计算比值
            if not after_result.empty and not before_result.empty:
                if before_result['success'].values[0] == 0:
                    algo_ratios[f'{x}'] = before_result['success'].values[0] - after_result['success'].values[0]  # 动态列名
                else:
                    algo_ratios[f'{x}'] = (before_result['success'].values[0] - after_result['success'].values[0]) / before_result['success'].values[0]  # 动态列名
            else:
                algo_ratios[f'{x}'] = 0  # 如果没有匹配的数据，则设为None
        
        result_df = result_df.append(algo_ratios, ignore_index=True)

    # print(result_df)
    numeric_df = result_df.select_dtypes(include='number')

    result_df['mean'] = numeric_df.mean(axis=1)
    # print(resul)
    result_df.to_csv(f'{save_name}.csv', index=False)
    print(result_df)
    

def print_final_performance_MetaWorld(df,x_values,save_name):
    # 计算每个组合的平均return
    avg_success = df.groupby(['algorithm', 'train task', 'test task'])['success'].mean().reset_index()

    result_df = pd.DataFrame(columns=['algorithm'])
    algorithms = avg_success['algorithm'].unique()
    train_mode = x_values[-1]
    for algo in algorithms:
        algo_data = avg_success[avg_success['algorithm'] == algo]
        
        algo_ratios = {'algorithm': algo}
        # 遍历每个x值
        for x in x_values:
            cur_condition = (avg_success['train task'] == x) & (avg_success['test task'] == x)
            
            # 筛选出train mode = x, test mode = x-1的行
            condition1 = (algo_data['train task'] == train_mode) & (algo_data['test task'] == x)
            result1 = algo_data[condition1]
            
            # 筛选出train mode = x-1, test mode = x-1的行
            prev_condition = (algo_data['train task'] == x) & (algo_data['test task'] == x)
            result2 = algo_data[prev_condition]
            
            # 如果两组数据都存在，则计算比值
            if not result1.empty and not result2.empty:
                algo_ratios[f'{x}'] = result1['success'].values[0]  # 动态列名
            else:
                algo_ratios[f'{x}'] = 0  # 如果没有匹配的数据，则设为None
        
        result_df = result_df.append(algo_ratios, ignore_index=True)

    # print(result_df)
    numeric_df = result_df.select_dtypes(include='number')

    result_df['mean'] = numeric_df.mean(axis=1)
    # print(resul)
    result_df.to_csv(f'{save_name}.csv', index=False)
    print(result_df)

In [27]:
# import pandas as pd

# # 读取CSV文件
# # df = pd.read_csv('eval_Freeway/eval_results.csv')
# # for x in range(1,9):
# #     x_values = range(x)
# #     print_final_performance(df, x_values, f'eval_Freeway/{x}_final_perf_results')

# df = pd.read_csv('eval_SpaceInvaders/eval_results.csv')
# for x in range(1,11):
#     x_values = range(x)
#     print_final_performance(df, x_values, f'eval_SpaceInvaders/{x}_final_perf_results')

# # df = pd.read_csv('eval_MetaWorld/eval_results.csv')
# # print(df.keys())
# # for x in range(1,21):
# #     x_values = range(x)
# #     print_final_performance_MetaWorld(df, x_values, f'eval_MetaWorld/{x}_final_perf_results')

In [30]:
import pandas as pd

# 读取CSV文件
# for x in range(1,9):
#     x_values = range(x)
#     print_final_performance(df, x_values, f'eval_Freeway/{x}_final_perf_results')

# df = pd.read_csv('eval_SpaceInvaders/eval_results.csv')
# for x in range(1,11):
#     x_values = range(x)
#     print_final_performance(df, x_values, f'eval_SpaceInvaders/{x}_final_perf_results')

# df = pd.read_csv('eval_MetaWorld/eval_results.csv')
# print(df.keys())
# for x in range(1,21):
#     x_values = range(x)
#     print_ratio_MetaWorld(df, x_values, f'eval_MetaWorld/{x}_forgetting_results')

# df = pd.read_csv('eval_Freeway/eval_results.csv')
# print_ratio(df, range(0,8), f"forgetting", last_mode=7)
df = pd.read_csv('eval_SpaceInvaders/eval_results.csv')
print_ratio(df, range(0,10), f"forgetting", last_mode=9)

  algorithm  forgetting on 0  forgetting on 1  forgetting on 2  \
0    CReLUs             0.08             0.63            -0.07   
1    CbpNet             0.39             0.45            -0.15   
2  CompoNet             0.22             0.46            -0.09   
3        F1             0.20             0.29            -0.02   
4        FN             0.58             0.55             0.05   
5   FuseNet             0.48             0.46            -0.28   
6   MaskNet            -0.29            -0.00            -0.08   
7   PackNet             0.09            -0.05             0.03   
8   ProgNet             0.24             0.25             0.44   

   forgetting on 3  forgetting on 4  forgetting on 5  forgetting on 6  \
0             0.06             0.28             0.20             0.49   
1             0.22             0.18             0.12             0.06   
2             0.10             0.48             0.06             0.30   
3             0.15             0.78            

<ipython-input-26-cde5e406cc13>:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(algo_ratios, ignore_index=True)
<ipython-input-26-cde5e406cc13>:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(algo_ratios, ignore_index=True)
<ipython-input-26-cde5e406cc13>:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(algo_ratios, ignore_index=True)
<ipython-input-26-cde5e406cc13>:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(algo_ratios, ignore_index=True)
<ipython-input-26-cde5e406cc13>:37: FutureWarning: The frame.append meth